<a href="https://colab.research.google.com/github/princedl/collab-java/blob/master/Shazam.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!sudo apt install ffmpeg

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 18 not upgraded.


In [2]:
!pip3 install ShazamAPI
!pip3 install pydub

In [ ]:
from pathlib import Path
import pickle

In [ ]:
!ffmpeg -i drive/MyDrive/DJAM/SCE\ RADIO\ -\ ALL\ THEM\ 80s\ -\ Jeff\ Scott\ Gould.m4a drive/MyDrive/DJAM/SCE\ RADIO\ -\ ALL\ THEM\ 80s\ -\ Jeff\ Scott\ Gould.mp3

ffmpeg version 3.4.11-0ubuntu0.1 Copyright (c) 2000-2022 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.5.0-3ubuntu1~18.04)
  configuration: --prefix=/usr --extra-version=0ubuntu0.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librubberband --enable-librsvg --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvorbis --enable-libvpx --enable-libwavpack --enable-libwebp --enable-libx265 --enable-libxml2 --enable-libxvid --enable-li

In [ ]:
fn = 'SCE-RADIO-ALL-THEM-80s-JeffScottGould.mp3'
fp = Path('drive/MyDrive/DJAM') / fn
sfp = str(fp)

In [ ]:
sfp

'drive/MyDrive/DJAM/SCE-RADIO-ALL-THEM-80s-JeffScottGould.mp3'

In [ ]:
!cp {sfp} .

In [ ]:
!rm out*.mp3

rm: cannot remove 'out*.mp3': No such file or directory


In [ ]:
!ffmpeg -i {fn} -f segment -segment_time 300 -c copy out%03d.mp3

ffmpeg version 3.4.11-0ubuntu0.1 Copyright (c) 2000-2022 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.5.0-3ubuntu1~18.04)
  configuration: --prefix=/usr --extra-version=0ubuntu0.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librubberband --enable-librsvg --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvorbis --enable-libvpx --enable-libwavpack --enable-libwebp --enable-libx265 --enable-libxml2 --enable-libxvid --enable-li

In [ ]:
from pydub import AudioSegment
from io import BytesIO
import requests
import uuid
import time
import json


from ShazamAPI.algorithm import SignatureGenerator
from ShazamAPI.signature_format import DecodedMessage

LANG = 'de'
TIME_ZONE = 'Europe/Berlin'


class Endpoint:
    SCHEME = 'https'
    HOSTNAME = 'amp.shazam.com'

    def __init__(
        self,
        lang: str,
        time_zone: str
    ) -> None:
        self.lang = lang
        self.time_zone = time_zone

    @property
    def url(self) -> str:
        return (
            f'{self.SCHEME}://{self.HOSTNAME}'
            '/discovery/v5'
            f'/{self.lang}/{self.lang.upper()}'
            '/iphone/-/tag/{uuid_a}/{uuid_b}'
        )

    @property
    def params(self) -> dict:
        return {
            'sync': 'true',
            'webv3': 'true',
            'sampling': 'true',
            'connected': '',
            'shazamapiversion': 'v3',
            'sharehub': 'true',
            'hubv5minorversion': 'v5.1',
            'hidelb': 'true',
            'video': 'v3'
        }

    @property
    def headers(self) -> dict:
        return {
            "X-Shazam-Platform": "IPHONE",
            "X-Shazam-AppVersion": "14.1.0",
            "Accept": "*/*",
            "Accept-Language": self.lang,
            "Accept-Encoding": "gzip, deflate",
            "User-Agent": "Shazam/3685 CFNetwork/1197 Darwin/20.0.0"
        }


class Shazam:
    MAX_TIME_SECONDS = 8

    def __init__(
        self,
        songData: bytes,
        lang: str = LANG,
        time_zone: str = TIME_ZONE
    ):
        self.songData = songData
        self._endpoint = Endpoint(lang, time_zone)

    def recognizeSong(self) -> dict:
        self.audio = self.normalizateAudioData(self.songData)
        signatureGenerator = self.createSignatureGenerator(self.audio)
        while True:

            signature = signatureGenerator.get_next_signature()
            if not signature:
                break

            results = self.sendRecognizeRequest(signature)
            currentOffset = signatureGenerator.samples_processed / 16000

            yield currentOffset, results

    def sendRecognizeRequest(self, sig: DecodedMessage) -> dict:
        data = {
            'timezone': self._endpoint.time_zone,
            'signature': {
                'uri': sig.encode_to_uri(),
                'samplems':int(sig.number_samples / sig.sample_rate_hz * 1000)
                },
            'timestamp': int(time.time() * 1000),
            'context': {},
            'geolocation': {}
                }
        r = requests.post(
            self._endpoint.url.format(
                uuid_a=str(uuid.uuid4()).upper(),
                uuid_b=str(uuid.uuid4()).upper()
            ),
            params=self._endpoint.params,
            headers=self._endpoint.headers,
            json=data
        )
        return r.json()

    def normalizateAudioData(self, songData: bytes) -> AudioSegment:
        audio = AudioSegment.from_file(BytesIO(songData))

        audio = audio.set_sample_width(2)
        audio = audio.set_frame_rate(16000)
        audio = audio.set_channels(1)
        return audio

    def createSignatureGenerator(self, audio: AudioSegment) -> SignatureGenerator:
        signature_generator = SignatureGenerator()
        signature_generator.feed_input(audio.get_array_of_samples())
        signature_generator.MAX_TIME_SECONDS = self.MAX_TIME_SECONDS
        if audio.duration_seconds > 12 * 3:
            signature_generator.samples_processed += 16000 * (int(audio.duration_seconds / 16) - 6)
        return signature_generator

In [ ]:
def recognize(fn):
  mp3_file = open(fn, 'rb')
  mp3_file_content_to_recognize = mp3_file.read()

  shazam = Shazam(
      mp3_file_content_to_recognize #,
      #lang='en',
      #time_zone='Europe/Paris'
  )
  recognize_generator = shazam.recognizeSong()
  # while True:
  # 	print(next(recognize_generator)) # current offset & shazam response to recognize requests

  rs = list()
  for m in recognize_generator:
    title = subtitle = '*'
    if 'track' in m[1]:
      title = m[1]['track']['title']
      subtitle = m[1]['track']['subtitle']
    r = (fn, m[0], title, subtitle)
    rs.append(r)
    print(r)

  mp3_file.close()
  return rs


In [ ]:
#recognize('out005.mp3')

In [ ]:
import glob
fns = glob.glob("./out*.mp3")
fns.sort()
rs = list()
for fn in fns:
  print(fn)
  rs.extend(recognize(fn))
rs

./out000.mp3
('./out000.mp3', 20.0, '99 Luftballons', 'Nena')
('./out000.mp3', 28.0, '99 Luftballons', 'Nena')
('./out000.mp3', 36.0, '99 Luftballons', 'Nena')
('./out000.mp3', 44.0, '99 Luftballons', 'Nena')
('./out000.mp3', 52.0, '99 Luftballons', 'Nena')
('./out000.mp3', 60.0, '99 Luftballons', 'Nena')
('./out000.mp3', 68.0, '99 Luftballons', 'Nena')
('./out000.mp3', 76.0, '99 Luftballons', 'Nena')
('./out000.mp3', 84.0, '*', '*')
('./out000.mp3', 92.0, '99 Luftballons', 'Nena')
('./out000.mp3', 100.0, '99 Luftballons', 'Nena')
('./out000.mp3', 108.0, '*', '*')
('./out000.mp3', 116.0, 'Paul Revere', 'Beastie Boys')
('./out000.mp3', 124.0, 'Paul Revere', 'Beastie Boys')
('./out000.mp3', 132.0, 'Paul Revere', 'Beastie Boys')
('./out000.mp3', 140.0, 'Paul Revere', 'Beastie Boys')
('./out000.mp3', 148.0, 'Sledgehammer', 'Peter Gabriel')
('./out000.mp3', 156.0, 'Sledgehammer', 'Peter Gabriel')
('./out000.mp3', 164.0, 'Sledgehammer', 'Peter Gabriel')
('./out000.mp3', 172.0, 'Sledgehammer'

[('./out000.mp3', 20.0, '99 Luftballons', 'Nena'),
 ('./out000.mp3', 28.0, '99 Luftballons', 'Nena'),
 ('./out000.mp3', 36.0, '99 Luftballons', 'Nena'),
 ('./out000.mp3', 44.0, '99 Luftballons', 'Nena'),
 ('./out000.mp3', 52.0, '99 Luftballons', 'Nena'),
 ('./out000.mp3', 60.0, '99 Luftballons', 'Nena'),
 ('./out000.mp3', 68.0, '99 Luftballons', 'Nena'),
 ('./out000.mp3', 76.0, '99 Luftballons', 'Nena'),
 ('./out000.mp3', 84.0, '*', '*'),
 ('./out000.mp3', 92.0, '99 Luftballons', 'Nena'),
 ('./out000.mp3', 100.0, '99 Luftballons', 'Nena'),
 ('./out000.mp3', 108.0, '*', '*'),
 ('./out000.mp3', 116.0, 'Paul Revere', 'Beastie Boys'),
 ('./out000.mp3', 124.0, 'Paul Revere', 'Beastie Boys'),
 ('./out000.mp3', 132.0, 'Paul Revere', 'Beastie Boys'),
 ('./out000.mp3', 140.0, 'Paul Revere', 'Beastie Boys'),
 ('./out000.mp3', 148.0, 'Sledgehammer', 'Peter Gabriel'),
 ('./out000.mp3', 156.0, 'Sledgehammer', 'Peter Gabriel'),
 ('./out000.mp3', 164.0, 'Sledgehammer', 'Peter Gabriel'),
 ('./out000.m

In [ ]:
rs

NameError: ignored

In [ ]:
pf = str(fp)+'.pkl'
print(pf)

drive/MyDrive/DJAM/KevinScott-LaborDay-2011.mp3.pkl


In [ ]:
file = open(pf, 'wb')
pickle.dump(rs, file)
file.close()

In [ ]:
file = open('drive/MyDrive/DJAM/KevinScott-LaborDay-2011.mp3.pkl', 'rb')
rs_ng = pickle.load(file)
file.close()

In [ ]:
chops=0
fn = rs_ng[0][0]
for r in rs_ng:
  if r[0] != fn:
    chops += 1
    fn = r[0]
  t = int(chops*300+r[1])
  print(f'{t//60}:{t%60}/{r}')


0:20/('./out000.mp3', 20.0, 'Stand Back', 'Stevie Nicks')
0:28/('./out000.mp3', 28.0, 'Hymn', 'The Admirals')
0:36/('./out000.mp3', 36.0, 'Stand Back (Tracy Takes You Home Mix)', 'Stevie Nicks')
0:44/('./out000.mp3', 44.0, 'Stand Back (Tracy Takes You Home Mix)', 'Stevie Nicks')
0:52/('./out000.mp3', 52.0, 'Stand Back', 'Stevie Nicks')
1:0/('./out000.mp3', 60.0, 'Stand Back (Morgan Page Edit)', 'Stevie Nicks')
1:8/('./out000.mp3', 68.0, 'Stand Back', 'Stevie Nicks')
1:16/('./out000.mp3', 76.0, 'Stand Back', 'Stevie Nicks')
1:24/('./out000.mp3', 84.0, '*', '*')
1:32/('./out000.mp3', 92.0, "Somebody's Baby", 'Jackson Browne')
1:40/('./out000.mp3', 100.0, "Somebody's Baby", 'Jackson Browne')
1:48/('./out000.mp3', 108.0, "Somebody's Baby", 'Jackson Browne')
1:56/('./out000.mp3', 116.0, "Somebody's Baby", 'Jackson Browne')
2:4/('./out000.mp3', 124.0, "Somebody's Baby", 'Jackson Browne')
2:12/('./out000.mp3', 132.0, '*', '*')
2:20/('./out000.mp3', 140.0, "Somebody's Baby", 'Jackson Browne')
